# ANALISE MATRIZES(BÁSICO) E SISTEMAS LINEARES

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm
import sympy as sp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
)
TRANSFORMATIONS = standard_transformations + (implicit_multiplication_application,)

def menu():
    while True:
        print('\n=== menu principal ===')
        print('1 - análise de matrizes')
        print('2 - sistema linear')
        print('3 - análise vetorial')
        print('4 - algebra geométrica(GA)')
        print('0 - sair')

        escolha = input('escolha uma opção:')

        if escolha == '1':
            submenuanalisematrizes()
        elif escolha == '2':
            submenusistemalinear()
        elif escolha == '3':
            print('em andamento...')
        elif escolha == '4':
            print('em andamento...')
        elif escolha == '0':
            print('saindo...')
            break
        else:
            print('opção inválida')

def menucomutividade(A1):
    print("\n você deseja verificar se a matriz comuta com outra matriz?")
    print('(1) sim')
    print('(0) não, voltar ao menu principal')

    escolhacomu = input('digite sua escolha:')

    if escolhacomu == '1':
        try:
            # Cria nova matriz B com input
            linhasB = int(input("Digite o número de linhas da matriz B: "))
            colunasB = int(input("Digite o número de colunas da matriz B: "))

            B = []
            print("Digite os elementos da matriz B linha por linha:")
            for i in range(linhasB):
                linha = input(f"Linha {i+1}: ")
                B.append([float(num) for num in linha.strip().split()])

            B = np.array(B)

            print("\nMatriz B:")
            print(B)

            # Verificar comutatividade
            if A1.shape[1] != B.shape[0] or B.shape[1] != A1.shape[0]:
                print("\nAs dimensões das matrizes não são compatíveis para verificação de comutatividade.")
            elif np.allclose(A1 @ B, B @ A1):
                print("\nA matriz comuta com B (pode ter observáveis simultâneos)")
            else:
                print("\nA matriz não comuta com B")

        except Exception as e:
            print("\nErro na verificação de comutatividade:", e)
    
    elif escolhacomu == '0':
        print("\nRetornando ao menu principal...")
        menu()  # ou chame o menu principal da forma correta no seu código
    
    else:
        print("Opção inválida.")
        menu_comutatividade(A1)

    


def analisematrizes():
    linhas = int(input("linhas:"))
    colunas = int(input('colunas:'))

    print("digite os elementos, linha por linha, separados por espaço:")
    matriz=[]
    for i in range(linhas):
        linha = input(f'linha {i+1}: ').split()
        linha = [float(x) for x in linha]
        matriz.append(linha)

    A1 = np.array(matriz)

    print('\n=== informações básicas: ===')
    print(A1)

    rank = np.linalg.matrix_rank(A1)

    print(f'rank: {rank:.3f}')

    #operações para matriz quadradas:   
    if linhas == colunas:
        print('\n=== operações básicas ===')

        try:
            inversa = np.linalg.inv(A1)
            print('\ninversa:')
            print(inversa)
        except np.linalg.LinAlgError:
            print('\ninversa: não existe(matriz singular)')

    
        determinante = np.linalg.det(A1)
        traco = np.trace(A1)
        autovalores, autovetores = np.linalg.eig(A1)
        # outras propriedades para matrizes quadradas


        print(f'determinante: {determinante:.3f}')
        print(f'traco: {traco:.3f}')
        print('\nautovalores:')
        print(autovalores)
        print('\nautovetores (em colunas):')
        print(autovetores)

    

        
    # visualizar autovetores da matriz A1
    if A1.shape == (2, 2):  # só plota se for 2x2
        plt.figure(figsize=(6, 6))
        plt.axhline(0, color='gray', lw=0.5)
        plt.axvline(0, color='gray', lw=0.5)

        origin = np.zeros(2)  # ponto (0,0)
    
        for i in range(2):
            vec = autovetores[:, i]
            val = autovalores[i]
            plt.quiver(*origin, *vec, angles='xy', scale_units='xy', scale=1, color='blue', label=f'λ = {val:.2f}')

        plt.title('Autovetores da matriz A1')
        plt.xlim(-2, 2)
        plt.ylim(-2, 2)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True)
        plt.legend()
        plt.gca().set_aspect('equal')
        plt.tight_layout()
        plt.show()


    #autovetores em 3d, se for 3x3:

    if A1.shape == (3, 3):
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')

        colors = ['blue', 'red', 'green']

        # Calcular maior norma para ajustar escala do gráfico
        max_norm = max(norm(autovetores[:, i]) * abs(autovalores[i]) for i in range(3))

        for i in range(3):
            vec = autovetores[:, i]
            val = autovalores[i]
            scaled_vec = vec * abs(val)  # escalar pelo autovalor (modulo)

            ax.quiver(
                0, 0, 0,
                scaled_vec[0], scaled_vec[1], scaled_vec[2],
                color=colors[i],
                arrow_length_ratio=0.1,
                label=f'λ = {0.0 if abs(val) < 1e-10 else val:.2f}'
            )

        # Ajustar os limites dinamicamente com base nos vetores
        lim = max(1, max_norm)
        ax.set_xlim([-lim, lim])
        ax.set_ylim([-lim, lim])
        ax.set_zlim([-lim, lim])

        # Rótulos e título
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_title('Autovetores da matriz A1 (3D)')
        ax.legend()
        plt.tight_layout()
        plt.show()


    else:('\n a matriz não é quadrada, então não tem determinante, traço ou autovalores')

    #código para matrizes singulares:

    if A1.shape[0] == A1.shape[1]:
        print('\n=== análise para matrizes singulares, analises de singularidade e estabilidade ===')

        detA1 = np.linalg.det(A1)
        cond = np.linalg.cond(A1)

        print(f'determinante: {detA1:.6f}')
        print(f'numero de condição: {cond:.2e}')

        if np.isclose(detA1, 0, atol=1e-10):
            print('matriz é quase singular, determinante ~ 0')
        if cond > 1e8:
            print('matriz mal condicionada, pode gerar erros numéricos')

    #gráfico da matriz A1 como heatmap
    print('\n=== heatmap da matriz A1 ===')
    plt.figure(figsize=(6,5))
    sns.heatmap(A1, annot=True, cmap='coolwarm', fmt='.2f', square=True, cbar_kws={'label': 'valor'})
    plt.title('matriz A1 - heatmap')
    plt.xlabel('colunas')
    plt.ylabel('linhas')
    plt.tight_layout()

    plt.show()
    
    #outras propriedades para essa matriz.
    print('\n=== propriedades e verficações estruturais ===')
    
    print(f'\nNorma de frobenius: {np.linalg.norm(A1, ord="fro"):.4f}')
    
    norm_espectral = np.linalg.norm(A1, 2)
    print(f"Norma espectral: {norm_espectral:.4f}")



    print('\nTransposta conjugada (hermitiana):')
    print(A1.conj().T)

    print('\nVerificações:')
    
    if A1.shape[0] == A1.shape[1]:
        if np.allclose(A1, A1.T):
            print("\nMatriz simétrica")
        else:
            print("\nMatriz não é simétrica")
    else:
        print("\nNão é uma matriz quadrada, logo não pode ser simétrica")

    
    if A1.shape[0] == A1.shape[1]:
        if np.allclose(A1, A1.conj().T):
            print("\nMatriz hermitiana")
        else:
            print("\nMatriz não é hermitiana")
    else:
        print("\nNão é uma matriz quadrada, logo não pode ser hermitiana")

    if A1.shape[0] == A1.shape[1] and np.allclose(A1.T @ A1, np.eye(A1.shape[0])):
        print('\nmatriz ortogonal')
    else:
        print('\nmatriz não é ortogonal')

    try:
        eigvals, eigvecs = np.linalg.eig(A1)
        if np.linalg.matrix_rank(eigvecs) == A1.shape[0]:
            print("\nA matriz é diagonalizável")
        else:
            print("\nA matriz não é diagonalizável")
    except Exception as e:
        print('\nNão é uma matriz quadrada, logo não é diagonalizável')
    

        if A1.ndim == 2 and A1.shape[0] == A1.shape[1]:  # matriz quadrada
            if np.allclose(A1 @ A1, A1):
                print("\nA matriz é idempotente")
            else:
                print("\nA matriz não é idempotente")
        else:
            print("\nMatriz não é quadrada ou não é 2D, logo não é idempotente")
    
    try:
        if np.allclose(A1 @ A1, np.eye(A1.shape[0])):
            print("\nA matriz é involutiva")
        else:
            print("\nA matriz não é involutiva")
    except Exception as e:
        print('\nNão é uma matriz quadrada, logo não é involutiva')


    try:
        if np.all(np.linalg.eigvals(A1) > 0):
            print("\nMatrix é positiva definida")
        else:
            print("\nMatris não é positiva definida")
    except Exception as e:
        print('\nnão é uma matriz quadrada, logo não é positiva definida')

    print('\n=== SVD ===')
    #SVD (singular value decomposition)
    U,S,Vt = np.linalg.svd(A1)
    print("\nSVD:")
    print("U =\n", U)
    print("S =\n", S)
    print("Vᵀ =\n", Vt)

    #exponencial da matriz
    print('\n=== exponencial da matriz ===')
    if A1.shape[0] == A1.shape[1]:
        try:
            from scipy.linalg import expm
            print('\nExponencial da matriz (e^A):')
            print(expm(A1))
        except Exception as e:
            print('\nNão é uma matriz quadrada')


    #polinomio carateristico
    print('\n=== polinomio caracteristico ===')
    try:
        from sympy import matrix, symbols, eye
        λ = symbols('λ')
        A_sym = Matrix(A1)
        p_char = A_sym.charpoly(λ)
        print(f"\nPolinômio característico: {p_char.as_expr()}")
    except Exception as e:
        print("\nNão foi possível calcular o polinômio característico (verifique se a matriz é quadrada).")

    print('\n === propriedades e operações uteis em física avançada(nuclear, quântica, etc) ===')

    # vamos usar essa flag pra saber se conseguimos calcular autovalores
    eigvals = None

    # --- espalhamento, espectro real, densidade, condição espectral ---
    if A1.shape[0] == A1.shape[1]:
        try:
            eigvals = np.linalg.eigvals(A1)

            # espalhamento
            spread = np.max(eigvals) - np.min(eigvals)
            print(f'\nEspalhamento (máx diferença entre autovalores): {spread:.4f}')

            # espectro real ou não
            print('\n -- verifica se tem espectro real ou não --')
            if np.all(np.isreal(eigvals)):
                print("\nTodos os autovalores são reais")
            else:
                print("\nMatriz possui autovalores complexos")

            # matriz de densidade (traço = 1 e autovalores >= 0)
            print('\n -- verifica se é uma matriz densidade --')
            if np.allclose(np.trace(A1), 1) and np.all(eigvals >= 0):
                print('pode ser uma matriz densidade')
            else:
                print('não satisfaz as condições de matriz densidade (traço = 1 e autovalores ≥ 0)')

            # condicionamento espectral
            print('\n -- condicionamento espectral --')
            if np.all(eigvals != 0):
                cond_esp = np.max(np.abs(eigvals)) / np.min(np.abs(eigvals))
                print(f"Número de condição espectral: {cond_esp:.4e}")
            else:
                print("Número de condição espectral indefinido (autovalor zero).")

        except Exception as e:
            print('\nnão foi possível calcular autovalores / grandezas espectrais (verifique se a matriz é quadrada e bem condicionada).')
            eigvals = None
    else:
        print('\nEspalhamento, espectro, matriz densidade e número de condição espectral só fazem sentido para matrizes quadradas.')

    # --- unitariedade ---
    print('\n -- unitariedade --')
    if A1.shape[0] == A1.shape[1]:
        try:
            U = A1
            if np.allclose(U.conj().T @ U, np.eye(U.shape[0])):
                print('\nA matriz é unitária')
            else:
                print('\nA matriz não é unitária')
        except Exception as e:
            print('\nnão foi possível verificar unitariedade:', e)
    else:
        print('\nMatriz não é quadrada, logo não pode ser unitária.')

    # --- observável (hermitiana) ---
    print('\n -- observável --')
    if A1.shape[0] == A1.shape[1]:
        try:
            if np.allclose(A1, A1.conj().T):
                print('\nmatriz representa um observável (hermitiana)')
            else:
                print('\nmatriz não é hermitiana')
        except Exception as e:
            print('\nnão foi possível testar se é hermitiana:', e)
    else:
        print('\nMatriz não é quadrada, logo não pode ser hermitiana.')

    # --- normalidade ---
    print('\n -- normalidade --')
    if A1.shape[0] == A1.shape[1]:
        try:
            if np.allclose(A1 @ A1.conj().T, A1.conj().T @ A1):
                print('matriz é normal')
            else:
                print('matriz não é normal')
        except Exception as e:
            print('\nnão foi possível verificar normalidade:', e)
    else:
        print('Normalidade só é definida para matrizes quadradas.')

    # --- comutatividade com outra matriz (teu menu especial) ---
    print('\n -- comutatividade --')
    menucomutividade(A1)

        
    
        
    


#código para resolver sistemas lineares:
#Ax = b

def sistemalinear():
    print('\n --- solução do sistema Ax = b ---')

    linhas2 = int(input("linhas do sistema linear:"))
    colunas2 = int(input('colunas do sistema linear:'))

    print("digite os elementos do sistema linear, linha por linha, separados por espaço:")
    matrizsis=[]

    for i in range(linhas2):
        linha = input(f'linha {i+1}: ').split()
        linha = [float(x) for x in linha]
        matrizsis.append(linha)
    A2 = np.array(matrizsis)

    print('digite o vetor b(lado direito). separe por espaço:')
    entradab = input('b: ').split()
    try:
        b = np.array([float(x) for x in entradab])
        if b.shape[0] != A2.shape[0]:
            print('erro: o vetor b deve ter o mesmo numero de linhas da matriz A.')
        else:
            x = np.linalg.solve(A2,b)
            print('solução x do sistema:')
            print(x)
    except np.linalg.LinAlgError:

        print('não foi possivel resolver o sistema. (matriz pode ser singular).')

        print('\n -- soluções alternativas ---')

        try:
            x_pseuinv = np.linalg.pinv(A2) @ b
            print("solução via pseudo-inversa:")
            print(x_pseuinv)
        except Exception as e:
            print('erro ao usar pseudo-inversa:',e)

        try:
            x_lstsq, residuals, rank, s = np.linalg.lstsq(A2, b, rcond=None)
            print('solução via minimos quadrados (lstsq):')
            print(x_lstsq)
            if residuals.size > 0:
                print(f'resíduo (||Ax-b||²): {residuals[0]:.4e}')
            else:
                print('sistema compatível ou infinitas soluções (sem resíduo)')
        except Exception as e:
            print('erro ao usar lstsq:',e)

    autovalores, autovetores = np.linalg.eig(A2)

    print('autovalores:')
    print(autovalores)
    print('autovetores (em coluna):')
    print(autovetores)


    plt.figure(figsize=(6,5))
    sns.heatmap(A2, annot=True, cmap='coolwarm', fmt='.2f', square=True, cbar_kws={'label': 'valor'})
    plt.title('matriz A2 - heatmap')
    plt.xlabel('colunas')
    plt.ylabel('linhas')
    plt.tight_layout()
    plt.show()


    #checar se x tem duas dimensões (ou seja, x[0] e x[1])

    if x.ndim == 1 and x.shape[0] == 2:

        plt.figure()
        plt.quiver(0, 0, x[0], x[1], angles='xy', scale_units='xy', scale=1, color='green')
        plt.xlim(-1, 1)
        plt.ylim(-1, 1)
        plt.grid()
        plt.title('Solução x do sistema como vetor')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.gca().set_aspect('equal')
        plt.show()

    elif x.ndim == 1 and x.shape[0] == 3:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.quiver(0, 0, 0, x[0], x[1], x[2], color='green')
        ax.set_xlim([-1, 1])
        ax.set_ylim([-1, 1])
        ax.set_zlim([-1, 1])
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_title('Solução x do sistema (3D)')
        plt.show()



    else:
        print("Solução x não é um vetor 2D. Não é possível plotar com quiver.")



# Matrizes simbólicas, funções

In [2]:
def submenuanalisematrizes():
    while True:
        print('\n === analise de matrizes ==')
        print('1 - analise básica(numérica)')
        print('2 - analise simbólica(com funções)')
        print('0 - voltar ao menu')

        op = input('escolha uma opção')

        if op =='1':
            analisematrizes()
        elif op =='2':
            analisematrizes_simbolica()
        elif op =='0':
            break
        else:
            print('opção inválida')

In [3]:
def ler_matriz_simbolica(var_name, linhas, colunas):
    """Lê uma matriz simbólica de tamanho (linhas x colunas)."""
    var = sp.Symbol(var_name)
    matriz = []
    for i in range(linhas):
        linha = []
        for j in range(colunas):
            while True:
                expr_str = input(f'a[{i+1},{j+1}] = ')
                try:
                    expr = parse_expr(
                        expr_str,
                        local_dict={var_name: var},
                        transformations=TRANSFORMATIONS,
                    )
                    linha.append(expr)
                    break
                except Exception as e:
                    print(f"Expressão inválida ({e}). Tente novamente.")
        matriz.append(linha)
    return sp.Matrix(matriz)

In [4]:
import sympy as sp

def analisematrizes_simbolica():
    print('\n === analise simbólica de matrizes ===')

    var = input('nome da variavel(escolhe x, pelo amor): ').strip()
    if var == '':
        var = 'x'
    var_ = sp.Symbol(var)

    linhas = int(input('numero de linhas: '))
    colunas = int(input('numero de colunas: '))

    print(f'\ndigite cada elemento como expressão em {var} (*não aperte espaço para separar, ele separa automaticamente, só ir enfiando as funções))')
    print(f'exemplos: {var}**2, sin({var}), 3*{var} +1, etc.\n')

    matriz = []
    for i in range(linhas):
        linha = []
        for j in range(colunas):
            expr_str = input(f'a[{i+1}, {j+1}] = ')
            expr = sp.sympify(expr_str, locals={var:var_})
            linha.append(expr)
        matriz.append(linha)

    A3 = sp.Matrix(matriz)

    print('\nmatriz A3:')
    sp.pprint(A3)
    print('\ndimensão:', A3.shape)

    if A3.rows == A3.cols:
        print('\ndeterminante de A3:')
        detA3 = A3.det()
        sp.pprint(detA3)

        print('\n traço de A3:')
        trA3 = A3.trace()
        sp.pprint(trA3)
    else:
        print('\n matriz não é quadrada.')

    #derivar cada elemento
    print(f'\n derivada de A3 em relação a {var}:')
    A3_deri = A3.applyfunc(lambda f: sp.diff(f, var_))
    sp.pprint(A3_deri)

    #avaliar a matriz em um valor de variavel, A3(X) -> A3(x = float)
    resp = input(f'\n deseja avaliar A3  com algum valor de {var}? (s/n): ')
    if resp == 's':
        valor = float(input(f'valor para {var}: '))
        A3_eval = A3.subs(var_, valor)
        print(f'\n A3({var}) = {valor}): ')
        sp.pprint(A3_eval)

    #posto(rank)
    rankA3 = A3.rank()
    print('\nrank de A3:', rankA3)

    #polinômio caracteristico e autovalores(se for quadrada né krl)
    if A3.rows == A3.cols:
        lam = sp.Symbol('λ')

        print('\n polinomio caracteristico de A3 (em lambida hihihikkkkk):')
        charpolyA3 = A3.charpoly(lam).as_expr()
        sp.pprint(charpolyA3)

        print('\n autovalores de A(com multiplicidade):')
        eigvalsA3 = A3.eigenvals()
        for val, mult in eigvalsA3.items():
            print('  λ =', val, '(multiplicidade =', mult, ')')
    else:
        print('\n A3 não é quadrada, daí não tem polinomio nem autovalores (usualmente)')
    
    if A3.rows == A3.cols:
        try:
            A3_inv = sp.simplify(A3.inv())
            print('\n inversa de A3 (A3^{-1}):')
            sp.pprint(A3_inv)
        except Exception as e:
            print('\n A3 não é inversivel ou houve problemas ao calcular A^{-1}:',e)
    
    #ortogonalidade
    #estou testando ortognalidade à direita.
    if A3.rows == A3.cols:
        A3TA3 = sp.simplify(A3.T*A3)
        I = sp.eye(A3.rows)
        print('\n A3.T*A3:')
        sp.pprint(I)

    if sp.simplify(A3TA3 -I) == sp.zeros(A3.rows):
        print('\n A3 é ortogonal (A3.T*A3 = I)')
    else:
        print('\n A3 não é ortogonal')
    #outras propriedades e operações

     # integral indefinida
    print(f'\nIntegral indefinida de A3 em relação a {var}:')
    A3_int = A3.applyfunc(lambda f: sp.integrate(f, var_))
    sp.pprint(A3_int)

    # integral definida opcional
    resp_int_def = input(
        f'\nDeseja calcular a integral DEFINIDA de A3 em relação a {var}? (s/n): '
    ).strip().lower()

    if resp_int_def == 's':
        print(f'\nDigite os limites de integração em {var} (podem ser números ou expressões, ex: 0, pi/2, 2*pi).')
        a_str = input('Limite inferior a = ')
        b_str = input('Limite superior b = ')
        try:
            a = parse_expr(a_str, local_dict={var: var_}, transformations=TRANSFORMATIONS)
            b = parse_expr(b_str, local_dict={var: var_}, transformations=TRANSFORMATIONS)
            print(f'\nIntegral definida de A3 em relação a {var}, de a={a} até b={b}:')
            A3_int_def = A3.applyfunc(lambda f: sp.integrate(f, (var_, a, b)))
            sp.pprint(A3_int_def)
        except Exception as e:
            print('\nNão foi possível interpretar ou integrar com os limites informados:', e)

    
     # 6) autovalores e autovetores de A3
    if A3.rows == A3.cols:
        print('\nAutovalores de A3 (com multiplicidades):')
        eigvalsA3 = A3.eigenvals()
        for val, mult in eigvalsA3.items():
            print('  λ =', val, ' (multiplicidade =', mult, ')')

        print('\nAutovetores de A3 (e autovalores associados):')
        try:
            eigvectsA3 = A3.eigenvects()  # lista de tuplas (λ, mult, [v1, v2, ...])
            for val, mult, vects in eigvectsA3:
                print('\nλ =', val, ' (multiplicidade =', mult, ')')
                print('Autovetores associados:')
                for v in vects:
                    sp.pprint(v)
        except Exception as e:
            print('\nProblema ao calcular autovetores de A3:', e)
    else:
        print('\nA3 não é quadrada; autovalores/autovetores não são definidos do jeito usual.')

    #posto da transposta
    print('\nPosto de A3.T:', A3.T.rank())

    #núcleo(kernel/nullscape)
    print('\nNúcleo (nullspace) de A3:')
    nullA3 = A3.nullspace()
    if nullA3:
        for v in nullA3:
            sp.pprint(v)
    else:
        print('A3 só tem o vetor nulo no núcleo.')

    #imagem(espaço coluna)
    print('\nImagem (columnspace) de A3:')
    colA3 = A3.columnspace()
    for v in colA3:
        sp.pprint(v)

    #decomposição LU

    try:
        L, U, _ = A3.LUdecomposition()
        print('\nDecomposição LU:')
        print('L =')
        sp.pprint(L)
        print('U =')
        sp.pprint(U)
    except:
        print('\nLU não foi possível para essa matriz.')

    #decomposição QR
    try:
        Q, R = A3.QRdecomposition()
        print('\nDecomposição QR:')
        print('Q =')
        sp.pprint(Q)
        print('R =')
        sp.pprint(R)
    except:
        print('\nQR não foi possível para essa matriz.')

    #matriz diagonalizável

    print('\nA3 é diagonalizável?')
    try:
        P, D = A3.diagonalize()
        print('Sim. Forma diagonal D:')
        sp.pprint(D)
    except:
        print('Não é diagonalizável.')

    #matriz exponencial

    print('\nExponencial da matriz exp(A3):')
    try:
        expA3 = sp.exp(A3)
        sp.pprint(expA3)
    except:
        print('Não foi possível calcular exp(A3).')

    #polinomio mínima, eu tinha escrito matriz minima kkkk

    print('\nPolinômio mínimo de A3:')
    try:
        minpoly = A3.minimal_polynomial()
        sp.pprint(minpoly)
    except:
        print('Não foi possível calcular o polinômio mínimo.')

    #RREF

    print('\nForma escalonada reduzida por linhas (RREF) de A3:')
    rrefA3, pivots = A3.rref()
    sp.pprint(rrefA3)
    print('Pivôs nas colunas:', pivots)

    


    
    resp_comp = input('\nDeseja comparar A3 com outra matriz B simbólica? (s/n): ').strip().lower()
    if resp_comp != 's':
        print('\n pulando comparação')        
    else:
        print('\n=== LEITURA DA MATRIZ B ===')
    
    print(f'A matriz B terá as MESMAS dimensões de A: {linhas} x {colunas}')
    print(f'Use a MESMA variável {var} nas expressões.\n')

    print("\n--- MATRIZ B ---")
    B = ler_matriz_simbolica(var, linhas, colunas)

    print('\nMatriz B:')
    sp.pprint(B)

    # determinante, traço, posto de B
    detB = None
    trB = None
    if B.rows == B.cols:
        print('\nDeterminante de B:')
        detB = sp.simplify(B.det())
        sp.pprint(detB)

        print('\nTraço de B:')
        trB = sp.simplify(B.trace())
        sp.pprint(trB)
    else:
        print('\nB não é quadrada; sem determinante/traço usuais.')

    rankB = B.rank()
    print('\nPosto (rank) de B:', rankB)

    # COMPARAÇÕES
    print('\n=== COMPARAÇÃO A x B ===')

    # igualdade de matrizes (A - B)
    print('\nA - B:')
    diffAB = sp.simplify(A3 - B)
    sp.pprint(diffAB)
    if diffAB == sp.zeros(*A3.shape):
        print('\nA e B são simbolicamente iguais (A = B).')
    else:
        print('\nA e B NÃO são simbolicamente iguais (A ≠ B).')

    if A3.rows == A3.cols and B.rows == B.cols:
        # det(A) vs det(B)
        if detA3 is not None and detB is not None:
            print('\nExpressão det(A) - det(B):')
            sp.pprint(sp.simplify(detA3 - detB))
            if sp.simplify(detA3 - detB) == 0:
                print('Conclusão: det(A) = det(B) simbolicamente.')
            else:
                print('Conclusão: det(A) ≠ det(B) em geral (a não ser para valores específicos de parâmetros).')

        # tr(A) vs tr(B)
        if trA3 is not None and trB is not None:
            print('\nExpressão tr(A) - tr(B):')
            sp.pprint(sp.simplify(trA3 - trB))
            if sp.simplify(trA3 - trB) == 0:
                print('Conclusão: tr(A3) = tr(B) simbolicamente.')
            else:
                print('Conclusão: tr(A) ≠ tr(B) em geral.')
    else:
        print('\nComo pelo menos uma matriz não é quadrada, não faz sentido comparar det/traço na forma usual.')

    # comparação de posto
    print('\nComparação de posto:')
    print(f'rank(A) = {rankA3},   rank(B) = {rankB}')
    if rankA3 == rankB:
        print('Conclusão: rank(A) = rank(B).')
    else:
        print('Conclusão: rank(A) ≠ rank(B).')


    #fazendo outras comparações com A3 e B

    print('\n=== COMPARAÇÃO A3 x B ===')

    # det(A3) = det(B)
    if detA3 is not None and detB is not None:
        resp_solve_det = input(
            f'\nDeseja resolver simbolicamente det(A3) = det(B) em função de {var}? (s/n): '
        ).strip().lower()

        if resp_solve_det == 's':
            eq_det = sp.Eq(detA3, detB)
            print('\nEquação det(A3) = det(B):')
            sp.pprint(eq_det)

            try:
                sol_det = sp.solve(eq_det, var_)
                print(f'\nSoluções de det(A3) = det(B) em relação a {var}:')
                sp.pprint(sol_det)
            except Exception as e:
                print('\nNão foi possível resolver det(A3) = det(B) simbolicamente:', e)

    # tr(A3) = tr(B)
    if trA3 is not None and trB is not None:
        resp_solve_tr = input(
            f'\nDeseja resolver simbolicamente tr(A3) = tr(B) em função de {var}? (s/n): '
        ).strip().lower()

        if resp_solve_tr == 's':
            eq_tr = sp.Eq(trA3, trB)
            print('\nEquação tr(A3) = tr(B):')
            sp.pprint(eq_tr)

            try:
                sol_tr = sp.solve(eq_tr, var_)
                print(f'\nSoluções de tr(A3) = tr(B) em relação a {var}:')
                sp.pprint(sol_tr)
            except Exception as e:
                print('\nNão foi possível resolver tr(A3) = tr(B) simbolicamente:', e)
    else:
        print('\nPelo menos uma das matrizes não é quadrada; det/traço não são comparáveis da forma usual.')

    print('\n --- sessão taylor + espectro ---')
    resp_taylor = input('deseja executar a sessão taylor + espectro com A3? (s/n): ')
    if resp_taylor == 's':
        taylor_espectro(A3)
    
    return A3

### Sessão Taylor + Espectro

Formalmente, definimos uma função analítica de matriz por meio de uma série de potências:

$$
f(A) = \sum_{n=0}^{\infty} c_n A^n
$$

Se a matriz $A$ for diagonalizável, isto é, se existir uma matriz inversível $P$ tal que

$$
A = P D P^{-1},
$$

onde $D$ é uma matriz diagonal contendo os autovalores de $A$, então vale o resultado fundamental:

$$
f(A) = P f(D) P^{-1},
$$

com

$$
f(D) = \operatorname{diag}\bigl(f(\lambda_1), f(\lambda_2), \dots \bigr).
$$

> **Teorema (Cálculo funcional para matrizes diagonalizáveis).**  
> Seja $A \in \mathbb{C}^{n \times n}$ uma matriz diagonalizável e seja
> $f(z) = \sum_{n=0}^{\infty} c_n z^n$ uma função analítica.
> Então
> $$
> f(A) = P f(D) P^{-1}.
> $$




In [5]:
def taylor_espectro(A):
    print('\n === sessaõ taylor + espectro ===')

    if A.rows != A.cols:
        print('a matriz não é quadrada')
        return

    n = A.rows
    print(f'matriz quadrada {n} x {n}')

    try:
        N = int(input('ordem N da expansão de taylor (ex: 5,10):'))
    except:
        print('entrada inválida para N')
        return

    print(f'\n contruindo EXP(A) via série de taylor truncada (N = {N})...')
    expA_taylor = sp.zeros(n)

    for k in range(N+1):
        expA_taylor += A**k / sp.factorial(k)

    expA_taylor = sp.simplify(expA_taylor)

    print('\n exp(A) aproximado por taylor:')
    sp.pprint(expA_taylor)


    #observações realistas para a próxima parte:
    #jordan simbólica pode falhar, limitação do sympy.
    # para matrizes grandes ou muito simbólicas, eingenvals() e jordan_form() podem ficar lentos
    # taylor truncado é otimo para didática e para casos numéricos; para simbólico, pode crescer rápido.

    print(f'\n--- Construindo exp(A), sin(A), cos(A) via Taylor truncada (N={N}) ---')

    I = sp.eye(n)
    expA_taylor = sp.zeros(n)
    sinA_taylor = sp.zeros(n)
    cosA_taylor = sp.zeros(n)

    Ak = I  # A^0
    for k in range(N + 1):
        # exp
        expA_taylor += Ak / sp.factorial(k)

        # cos: só termos pares
        if k % 2 == 0:
            m = k // 2
            cosA_taylor += ((-1)**m) * Ak / sp.factorial(k)

        # sin: só termos ímpares
        if k % 2 == 1:
            m = (k - 1) // 2
            sinA_taylor += ((-1)**m) * Ak / sp.factorial(k)

        Ak = Ak * A  # próximo A^(k+1)

    expA_taylor = sp.simplify(expA_taylor)
    sinA_taylor = sp.simplify(sinA_taylor)
    cosA_taylor = sp.simplify(cosA_taylor)

    print('\nexp(A) via Taylor:')
    sp.pprint(expA_taylor)
    print('\nsin(A) via Taylor:')
    sp.pprint(sinA_taylor)
    print('\ncos(A) via Taylor:')
    sp.pprint(cosA_taylor)

    print('\n--- Espectro de A ---')
    
    try:
        eigA = A.eigenvals()
        for val, mult in eigA.items():
            print(f'λ = {val} (multiplicidade = {mult})')
    except Exception as e:
        print('Não foi possível calcular eigenvals(A) simbolicamente:', e)
        eigA = None

    # espectro de exp(A) via Taylor
    print('\n--- Espectro de exp(A) (Taylor) ---')
    try:
        eig_expA = expA_taylor.eigenvals()   # <-- era eigenval() no seu código
        for val, mult in eig_expA.items():
            print(f'μ = {val} (multiplicidade = {mult})')
    except Exception as e:
        print('Não foi possível calcular eigenvals(expA_taylor) simbolicamente:', e)

    # comparação teórica exp(λ)
    if eigA is not None:
        print('\n--- Comparação teórica: exp(λ) para cada autovalor λ de A ---')
        for val in eigA.keys():
            print(f'exp({val}) = {sp.exp(val)}')

    print('\nObservação:')
    print('Se A for diagonalizável, os autovalores de exp(A) são exp(λ_i).')
    print('Diferenças podem ocorrer por truncamento (Taylor) ou simplificação simbólica.')

    print('\n--- Função genérica f(A) = Σ c_n A^n (truncada) ---')
    
    resp_f = input('Deseja construir f(A) genérica a partir de coeficientes c0..cN? (s/n): ').strip().lower()
    if resp_f == 's':
        print('Digite c0,c1,...,cN separados por vírgula.')
        print('Ex: exp -> 1, 1, 1/2, 1/6, 1/24, ...')
        s = input('Coeficientes: ').strip()
        try:
            parts = [p.strip() for p in s.split(',')]
            if len(parts) != N + 1:
                print(f'Você precisa fornecer exatamente {N+1} coeficientes.')
            else:
                coeffs = [sp.sympify(p) for p in parts]
                fA = sp.zeros(n)
                Ak = I
                for k in range(N + 1):
                    fA += coeffs[k] * Ak
                    Ak = Ak * A
                fA = sp.simplify(fA)
                print('\nf(A) genérica (Taylor por coeficientes):')
                sp.pprint(fA)
        except Exception as e:
            print('Falhou ao interpretar coeficientes/gerar f(A):', e) 

    
    
    print('\n--- Caso não diagonalizável (Jordan): f(A) via Jordan ---')
    
    resp_j = input('Deseja tentar Jordan (exp/sin/cos via Jordan)? (s/n): ').strip().lower()
    if resp_j == 's':
        z = sp.Symbol('z')
        try:
            P, J = A.jordan_form()
            print('\nJordan J:')
            sp.pprint(J)

            # construir f(J) por blocos: f(J)=Σ f^(k)(λ)/k! * N^k
            def f_of_jordan(f_expr):
                fJ = sp.zeros(n)

                # percorre blocos jordan (assumindo 1 na superdiagonal dentro do bloco)
                i = 0
                while i < n:
                    lam = sp.simplify(J[i, i])
                    m = 1
                    while i + m < n and sp.simplify(J[i + m - 1, i + m]) == 1:
                        m += 1

                    # bloco nilpotente N (m x m)
                    Nblk = sp.zeros(m)
                    for r in range(m - 1):
                        Nblk[r, r + 1] = 1

                    blk = sp.zeros(m)
                    Nk = sp.eye(m)
                    for k in range(m):
                        deriv = sp.diff(f_expr, z, k)
                        coeff = deriv.subs(z, lam) / sp.factorial(k)
                        blk += coeff * Nk
                        Nk = Nk * Nblk

                    fJ[i:i+m, i:i+m] = blk
                    i += m

                return sp.simplify(P * fJ * P.inv())

            print('\nexp(A) via Jordan:')
            sp.pprint(f_of_jordan(sp.exp(z)))
            print('\nsin(A) via Jordan:')
            sp.pprint(f_of_jordan(sp.sin(z)))
            print('\ncos(A) via Jordan:')
            sp.pprint(f_of_jordan(sp.cos(z)))

        except Exception as e:
            print('Jordan falhou (normal em matrizes muito simbólicas/grandes):', e)

    print("\n--- Aplicação física: x'(t)=A x ---")
    
    resp_ode = input("Deseja resolver x'(t)=A x com x(0)=x0 usando exp(A t) via Taylor? (s/n): ").strip().lower()
    if resp_ode == 's':
        t = sp.Symbol('t', real=True)
        print(f'Digite x0 com {n} componentes separados por vírgula. Ex: 1,0,0')
        s = input('x0: ').strip()
        try:
            x0_list = [sp.sympify(p.strip()) for p in s.split(',')]
            x0 = sp.Matrix(x0_list)
            if x0.rows != n or x0.cols != 1:
                print(f'x0 deve ser vetor coluna {n}x1.')
            else:
                # exp(A t) via Taylor com a MESMA ordem N
                expAt = sp.zeros(n)
                Ak = I
                for k in range(N + 1):
                    expAt += (t**k) * Ak / sp.factorial(k)
                    Ak = Ak * A
                expAt = sp.simplify(expAt)

                xt = sp.simplify(expAt * x0)
                print('\nexp(A t) (Taylor):')
                sp.pprint(expAt)
                print('\nSolução x(t)=exp(A t)x0 (Taylor):')
                sp.pprint(xt)
        except Exception as e:
            print('Falhou ao interpretar x0 ou resolver:', e)

    print('\n=== FIM DA SESSÃO TAYLOR + ESPECTRO ===')
    

In [6]:
#funções auxiliares

import sympy as sp

def _is_square(A):
    return hasattr(A, "rows") and hasattr(A, "cols") and A.rows == A.cols

def matrix_taylor_from_coeffs(A, coeffs, N, simplify=True):
    """
    f(A) ≈ Σ_{k=0..N} c_k A^k (truncada)
    coeffs pode ser:
      - função coeffs(k)->c_k
      - lista [c0, c1, ..., cN]
    """
    if not _is_square(A):
        raise ValueError("A deve ser quadrada.")
    n = A.rows
    S = sp.zeros(n)
    Ak = sp.eye(n)

    for k in range(N + 1):
        ck = coeffs(k) if callable(coeffs) else coeffs[k]
        S += ck * Ak
        Ak = Ak * A  # acumula A^k de forma mais eficiente

    return sp.simplify(S) if simplify else S

def exp_taylor(A, N):
    return matrix_taylor_from_coeffs(A, lambda k: 1/sp.factorial(k), N)

def sin_taylor(A, N):
    # sin(A) = Σ_{m>=0} (-1)^m A^(2m+1)/(2m+1)!
    def c(k):
        if k % 2 == 0:
            return 0
        m = (k - 1) // 2
        return (-1)**m / sp.factorial(k)
    return matrix_taylor_from_coeffs(A, c, N)

def cos_taylor(A, N):
    # cos(A) = Σ_{m>=0} (-1)^m A^(2m)/(2m)!
    def c(k):
        if k % 2 == 1:
            return 0
        m = k // 2
        return (-1)**m / sp.factorial(k)
    return matrix_taylor_from_coeffs(A, c, N)

def _jordan_blocks_from_J(J):
    """Extrai blocos de Jordan de uma matriz J (block diagonal)."""
    n = J.rows
    blocks = []
    i = 0
    while i < n:
        lam = sp.simplify(J[i, i])
        size = 1
        while i + size < n and sp.simplify(J[i + size - 1, i + size]) == 1:
            size += 1
        blocks.append((i, size, lam))
        i += size
    return blocks

def matrix_function_via_jordan(A, f_expr, z=None, simplify=True):
    """
    f(A) via forma de Jordan:
    Para cada bloco J = λI + N (N nilpotente), f(J)=Σ_{k=0..m-1} f^(k)(λ)/k! * N^k.
    """
    if not _is_square(A):
        raise ValueError("A deve ser quadrada.")
    z = z or sp.Symbol('z')
    f_expr = sp.sympify(f_expr)

    # tentar Jordan
    P, J = A.jordan_form()

    n = A.rows
    fJ = sp.zeros(n)
    blocks = _jordan_blocks_from_J(J)

    for start, m, lam in blocks:
        N = sp.zeros(m)
        for r in range(m - 1):
            N[r, r + 1] = 1

        block = sp.zeros(m)
        Nk = sp.eye(m)

        for k in range(m):
            deriv_k = sp.diff(f_expr, z, k)
            coeff_k = deriv_k.subs(z, lam) / sp.factorial(k)
            block += coeff_k * Nk
            Nk = Nk * N

        fJ[start:start+m, start:start+m] = block

    fA = P * fJ * P.inv()
    return sp.simplify(fA) if simplify else fA

def solve_xdot_Ax(A, x0, t=None, method="taylor", N=10):
    """
    x'(t)=A x, x(0)=x0  => x(t)=exp(A t) x0
    method: "taylor", "sympy", "jordan"
    """
    if not _is_square(A):
        raise ValueError("A deve ser quadrada.")
    n = A.rows
    t = t or sp.Symbol('t', real=True)

    x0 = sp.Matrix(x0)
    if x0.rows != n or x0.cols != 1:
        raise ValueError(f"x0 deve ser vetor coluna {n}x1.")

    if method == "sympy":
        expAt = sp.exp(A * t)
    elif method == "jordan":
        z = sp.Symbol('z')
        expAt = matrix_function_via_jordan(A, sp.exp(z*t), z=z)
    else:
        expAt = exp_taylor(A * t, N)

    return sp.simplify(expAt * x0)


In [ ]:
menu()


=== menu principal ===
1 - análise de matrizes
2 - sistema linear
3 - análise vetorial
4 - algebra geométrica(GA)
0 - sair


escolha uma opção: 1



 === analise de matrizes ==
1 - analise básica(numérica)
2 - analise simbólica(com funções)
0 - voltar ao menu


escolha uma opção 2



 === analise simbólica de matrizes ===


...


Jordan J:
⎡ sin(x)    sin(x - 1)⎤
⎣x       + ℯ          ⎦

exp(A) via Jordan:
⎡  sin(x)    sin(x - 1)⎤
⎢ x       + ℯ          ⎥
⎣ℯ                     ⎦

sin(A) via Jordan:
⎡   ⎛ sin(x)    sin(x - 1)⎞⎤
⎣sin⎝x       + ℯ          ⎠⎦

cos(A) via Jordan:
⎡   ⎛ sin(x)    sin(x - 1)⎞⎤
⎣cos⎝x       + ℯ          ⎠⎦

--- Aplicação física: x'(t)=A x ---


Deseja resolver x'(t)=A x com x(0)=x0 usando exp(A t) via Taylor? (s/n):  s


Digite x0 com 1 componentes separados por vírgula. Ex: 1,0,0


x0:  1,0,0


x0 deve ser vetor coluna 1x1.

=== FIM DA SESSÃO TAYLOR + ESPECTRO ===

 === analise de matrizes ==
1 - analise básica(numérica)
2 - analise simbólica(com funções)
0 - voltar ao menu
